In [ ]:
# !pip install tensorflow

In [ ]:
import pandas as pd

train_df = pd.read_csv('sample_data/california_housing_train.csv')
test_df = pd.read_csv('sample_data/california_housing_test.csv')

train_df.head()

In [ ]:
X_train_np = train_df.to_numpy()[:, :-1] # all rows, all columns except last (label -> house value)
y_train_np = train_df.to_numpy()[:, -1] # last column

X_train_np.shape, y_train_np.shape

In [ ]:
X_test_np = test_df.to_numpy()[:, :-1] # all rows, all columns except last (label -> house value)
y_test_np = test_df.to_numpy()[:, -1] # last column

X_test_np.shape, y_test_np.shape

In [ ]:
import torch
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(torch.tensor(X_train_np, dtype=torch.float),
                              torch.tensor(y_train_np.reshape((-1, 1)), dtype=torch.float))

# reshape (-1,1) means convert vector-row to vector-column

train_dataset

In [ ]:
# Convert TensorDataset to DataLoader

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=128) # 128 is ok since it is not images and DS is small

for X, y in train_dataloader:
    print(X.shape, y.shape)
    break

In [ ]:

test_dataset = TensorDataset(torch.tensor(X_test_np, dtype=torch.float),
                              torch.tensor(y_test_np.reshape((-1, 1)), dtype=torch.float))

# reshape (-1,1) means convert vector-row to vector-column

test_dataset

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=64) # smaller DS - batch is 64

for X, y in test_dataloader:
    print(X.shape, y.shape)
    break

In [ ]:
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        # Layers have to be specified explicitly
        self.hidden_layer_1 = nn.Linear(8, 64)  # input: 8, fully connected => output: 64 ?
        self.hidden_activation = nn.ReLU()  # hidden layer and activation are functions?
        
        self.out = nn.Linear(64, 1)
        
    # specify pipeline    
    def forward(self, x):
        x = self.hidden_layer_1(x)
        x = self.hidden_activation(x)
        x = self.out(x)
        return x
    
    
model = NeuralNet().to(device)   # link do device
print(model)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # lr == learning rate


In [ ]:
def train(dataloader, model, loss_fn, optimiser):
    model.train() # it is not actual training - just set model to training mode
    train_loss = 0 # will accumulate it
    
    for i, (X,y) in enumerate(dataloader): # do we really need i in this loop ? Unless we print per batch iteration
        X, y = X.to(device), y.to(device)  # link to device (gpu/cpu)
        
        y_hat = model(X)  # make prediction on train data
        mse = loss_fn(y_hat, y) # it is the tensor that stores computation of loss between pred an sctual 
        train_loss += mse.item() # number sinse it is one dimensional
        
        optimizer.zero_grad()  # zeroed gradients - in optimization
        mse.backward()              # back propagation
        optimiser.step()            # one step
        
    num_batches = len(dataloader)
    train_mse = train_loss / num_batches   # average over number of batches
    print(f'Train RMSE: {train_mse**(1/2)}')
        

In [ ]:
# As in TF we keeep switching between train and test

def test(dataloader, model, loss_fn):
    model.eval()  # set model in evaluation mode
    test_loss = 0
    # sinse it is test - no optimization, no derivateves calculating
    with torch.no_grad():
        for X, y in dataloader:  # no need for i index
            X, y = X.to(device), y.to(device)
            y_hat = model(X)
            test_loss += loss_fn(y_hat, y).item()  # since loss here is tensor with dimension > 1
                                                   # we convert it to value using item() call
    num_batches = len(dataloader)
    test_mse = test_loss / num_batches
    
    print(f'Test RMSE: {test_mse**(1/2)}\n')

In [ ]:
# As in TF we do something like fit -> train/test 

epochs = 10  # try 1000

for epoch in range(epochs):
    print(f'Epoch {epoch+1}:') # + 1 since epochs range is 0 based
    
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

In [ ]:
# NOTE we did not optimize hyperparameters!

### Now the same modeling using TensorFlow: (reworked)

In [ ]:
# Load and prepare data

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load the data
train_df = pd.read_csv('sample_data/california_housing_train.csv')
test_df = pd.read_csv('sample_data/california_housing_test.csv')

# Prepare the data
X_train_np = train_df.iloc[:, :-1].to_numpy().astype('float32')
y_train_np = train_df.iloc[:, -1].to_numpy().reshape((-1, 1)).astype('float32')

X_test_np = test_df.iloc[:, :-1].to_numpy().astype('float32')
y_test_np = test_df.iloc[:, -1].to_numpy().reshape((-1, 1)).astype('float32')

# Create TensorFlow datasets
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y_train_np))
# train_dataset = train_dataset.batch(128)

# test_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y_test_np))
# test_dataset = test_dataset.batch(64)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y_train_np))
train_dataset = train_dataset.batch(128, drop_remainder=True)  # Use drop_remainder to drop incomplete batches

test_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y_test_np))
test_dataset = test_dataset.batch(64, drop_remainder=True)  # Use drop_remainder to drop incomplete batches





In [ ]:
# Modeling and testing
# Define the model

model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_np.shape[1],)),
    keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training loop
epochs = 10 # 100

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}:')
    
    # Training
    train_loss = 0
    num_batches = 0
    
    for X, y in train_dataset:
        with tf.GradientTape() as tape:
            y_hat = model(X)
            loss = tf.keras.losses.mean_squared_error(y, y_hat)
        
        grads = tape.gradient(loss, model.trainable_variables)
        model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        train_loss += loss.numpy() # np.array(loss) 
        num_batches += 1
    
    train_rmse = tf.math.sqrt(train_loss / num_batches)
    print(f'Train RMSE: {train_rmse.numpy()}')
    
    # Testing
    test_loss = 0
    num_batches = 0
    
    for X, y in test_dataset:
        y_hat = model(X)
        loss = tf.keras.losses.mean_squared_error(y, y_hat)
        
        test_loss += loss.numpy()
        num_batches += 1
    
    test_rmse = tf.math.sqrt(test_loss / num_batches)
    print(f'Test RMSE: {test_rmse.numpy()}\n')